# Advanced Querying Mongo

Importing libraries and setting up connection

In [60]:
!pip3 install pymongo 
from pymongo import MongoClient
client = MongoClient("localhost:27017")
import pandas as pd

In [10]:
client.list_database_names()

['Ironhack', 'admin', 'config', 'local']

In [13]:
db = client.get_database("Ironhack")

In [14]:
db.list_collection_names()

['companies', 'books', 'restaurants', 'countries-small']

In [15]:
collection = db.get_collection("companies")

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [22]:
collection.find_one({}).keys()

dict_keys(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url', 'blog_url', 'blog_feed_url', 'twitter_username', 'category_code', 'number_of_employees', 'founded_year', 'founded_month', 'founded_day', 'deadpooled_year', 'tag_list', 'alias_list', 'email_address', 'phone_number', 'description', 'created_at', 'updated_at', 'overview', 'image', 'products', 'relationships', 'competitions', 'providerships', 'total_money_raised', 'funding_rounds', 'investments', 'acquisition', 'acquisitions', 'offices', 'milestones', 'video_embeds', 'screenshots', 'external_links', 'partners'])

In [85]:
Babelgum = collection.find({"name": {"$regex": ".*Babelgum*."}}, {"name": 1, "_id": 0})

In [86]:
pd.DataFrame(Babelgum)

,name
0,Babelgum


### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [44]:
employees = collection.find({"number_of_employees": {"$gt": 5000}},{"name": 1, "number_of_employees":1, "_id":0}).sort("number_of_employees", 1).limit(5)
pd.DataFrame(employees)

,name,number_of_employees
0,Nintendo,5080
1,Hexaware Technologies,5200
2,Facebook,5299
3,OpenText,5300
4,CPM Braxis,5400


### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [64]:
founded = collection.find({"founded_year": {"$gte": 2000} and {"$lte": 2005}},{"name": 1, "founded_year":1, "_id":0}).limit(5)
pd.DataFrame(founded)

,name,founded_year
0,Wetpaint,2005
1,AdventNet,1996
2,Zoho,2005
3,Digg,2004
4,Facebook,2004


### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [95]:
valuation = collection.find({"$and": [ {"ipo.valuation_amount": {"$gt": 100000000}}, {"founded_year": {"$lt":2010}}]}
                             ,{"name": 1, "ipo.valuation_amount":1, "_id":0})

                                                                                    
pd.DataFrame(valuation).head()

,name,ipo
0,Facebook,{'valuation_amount': 104000000000}
1,Twitter,{'valuation_amount': 18100000000}
2,Yelp,{'valuation_amount': 1300000000}
3,LinkedIn,{'valuation_amount': 9310000000}
4,Amazon,{'valuation_amount': 100000000000}


### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [76]:
empl_found = collection.find({"$and": [ {"number_of_employees": {"$lt": 1000}}, {"founded_year": {"$lt":2005}}]}
                             ,{"name": 1, "number_of_employees":1,"founded_year": 1, "_id":0}).sort("number_of_employees", 1).limit(10)
pd.DataFrame(empl_found).head()

,name,number_of_employees,founded_year
0,Fox Interactive Media,0,1979
1,Ticketmaster,0,1976
2,Eurekster,0,2004
3,MindTouch,0,2004
4,Skype,0,2003


### 6. All the companies that don't include the `partners` field.

In [130]:
nopartners = collection.find({"parters": {"$exists": False}}, {"name": 1, "_id":0})
pd.DataFrame(nopartners).head()

,name
0,Wetpaint
1,AdventNet
2,Zoho
3,Digg
4,Facebook


### 7. All the companies that have a null type of value on the `category_code` field.

In [111]:
nulls = collection.find({"category_code": {"$regex": ".*null*."}}, {"name": 1, "_id":0})
nulls

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [93]:
x = collection.find({"number_of_employees": {"$lt": 100} and {"$gt": 100}}
                             ,{"name": 1, "number_of_employees":1, "_id":0})
pd.DataFrame(x).head()

,name,number_of_employees
0,AdventNet,600
1,Zoho,1600
2,Facebook,5299
3,Twitter,1300
4,eBay,15000


### 9. Order all the companies by their IPO price in a descending order.

In [108]:
IPO = collection.find({"ipo.valuation_amount" : {"$gt":0}},{"name": 1, "ipo.valuation_amount": 1, "_id":0}).sort("ipo.valuation_amount",-1)
                           
                                                                                    
pd.DataFrame(IPO).head()

,name,ipo
0,GREE,{'valuation_amount': 108960000000}
1,Facebook,{'valuation_amount': 104000000000}
2,Amazon,{'valuation_amount': 100000000000}
3,Twitter,{'valuation_amount': 18100000000}
4,Groupon,{'valuation_amount': 12800000000}


### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [109]:
more_employees = collection.find({"ipo.valuation_amount" : {"$gt":0}},{"name": 1, "ipo.valuation_amount": 1, "_id":0}).sort("number_of_employees",1)
                                                                                                             
pd.DataFrame(more_employees).head()



,name,ipo
0,Amazon,{'valuation_amount': 100000000000}
1,JumpTV,{'valuation_amount': 100000000}
2,Nielsen,{'valuation_amount': 1600000000}
3,Vringo,{'valuation_amount': 11003200}
4,TripAdvisor,{'valuation_amount': 3273770000}


### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [114]:
semester = collection.find({"founded_month" : {"$gt":6}},{"name": 1, "founded_month": 1, "_id":0}).limit(1000)
                                                                                                             
pd.DataFrame(semester).head()

,name,founded_month
0,Wetpaint,10
1,Zoho,9
2,Digg,10
3,Omnidrive,11
4,eBay,9


### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [132]:
semester = collection.find(
    {"$and":[{"founded_year" : {"$lte":2000}}, {"acquisition.price_amount": {"$gt": 10000000}}]},
    {"name": 1, "founded_year": 1, "acquisition.price_amount":1, "_id":0})
                                                                                                             
pd.DataFrame(semester).head()

,name,founded_year,acquisition
0,Postini,1999,{'price_amount': 625000000}
1,SideStep,1999,{'price_amount': 180000000}
2,Recipezaar,1999,{'price_amount': 25000000}
3,PayPal,1998,{'price_amount': 1500000000}
4,Snapfish,1999,{'price_amount': 300000000}


### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [121]:
after2010 = collection.find({"acquisition.acquired_year" : {"$lt":2010}}, {"name": 1, "founded_year": 1, "acquisition.price_amount":1, "_id":0}).sort("acquisition.price_amount",1)
                                                                                                             
pd.DataFrame(after2010).head()

,name,founded_year,acquisition
0,Gizmoz,2003.0,{'price_amount': None}
1,Joost,2006.0,{'price_amount': None}
2,Gannett,NaN,{'price_amount': None}
3,JotSpot,NaN,{'price_amount': None}
4,Livestream,2007.0,{'price_amount': None}


### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [133]:
founded_year = collection.find({}, {"name": 1, "founded_year": 1, "_id":0}).sort("founded_year",1)
                                                                                                             
pd.DataFrame(founded_year).head()

,name,founded_year
0,Flektor,NaN
1,Lala,NaN
2,SpinVox,NaN
3,Gannett,NaN
4,Info,NaN


### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [124]:
day = collection.find({"founded_day" : {"$lte":7}},{"name": 1, "founded_day":1,"acquisition.price_amount":1, "_id":0}).sort("acquisition.price_amount", 1)
                                                                                                             
pd.DataFrame(day).head()

,name,founded_day,acquisition
0,Facebook,1,NaN
1,Omnidrive,1,NaN
2,Geni,1,{'price_amount': None}
3,Fox Interactive Media,1,NaN
4,Gizmoz,1,{'price_amount': None}


### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [137]:
web = collection.find({"$and": [{"category_code": "web"}, {"number_of_employees": {"$gt": 4000}}]}).sort("number_of_employees",1)
pd.DataFrame(web).head()

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,investments,acquisition,acquisitions,offices,milestones,ipo,video_embeds,screenshots,external_links,partners
0,52cdef7c4bab8bd67529822a,Expedia,expedia,http://www.crunchbase.com/company/expedia,http://www.expedia.com,,,Expedia,web,4400,...,"[{'funding_round': {'round_code': 'c', 'source...","{'price_amount': None, 'price_currency_code': ...","[{'price_amount': None, 'price_currency_code':...","[{'description': 'Corporate Office', 'address1...","[{'id': 11691, 'description': 'US AIRWAYS AND ...","{'valuation_amount': None, 'valuation_currency...",[],"[{'available_sizes': [[[150, 93], 'assets/imag...",[{'external_url': 'http://www.urlaubs-rabatte....,[]
1,52cdef7c4bab8bd675297e96,AOL,aol,http://www.crunchbase.com/company/aol,http://www.aol.com,http://blog.aol.com/,http://feeds.feedburner.com/AolBlog?format=xml,aol,web,8000,...,"[{'funding_round': {'round_code': 'b', 'source...",None,"[{'price_amount': 40000000, 'price_currency_co...","[{'description': 'HQ', 'address1': '770 Broadw...","[{'id': 2689, 'description': 'AOL relauches, c...","{'valuation_amount': None, 'valuation_currency...","[{'embed_code': '<embed src=""http://blip.tv/pl...","[{'available_sizes': [[[150, 91], 'assets/imag...","[{'external_url': 'http://mashpedia.com/AOL', ...",[]
2,52cdef7c4bab8bd675297ea4,Webkinz,webkinz,http://www.crunchbase.com/company/webkinz,http://www.webkinz.com,http://www.webkinz.com,http://piczo.com,webkinz,web,8657,...,[],None,[],"[{'description': None, 'address1': 'One Pearce...",[],None,[],[],"[{'external_url': 'http://webkinz-tips.com', '...",[]
3,52cdef7c4bab8bd675297fcb,Rakuten,rakuten,http://www.crunchbase.com/company/rakuten,http://global.rakuten.com/corp,,,RakutenGlobal,web,10000,...,[{'funding_round': {'round_code': 'unattribute...,None,"[{'price_amount': 425000000, 'price_currency_c...","[{'description': 'Headquarter', 'address1': 'R...","[{'id': 7034, 'description': 'RS Empowerment a...",None,[],"[{'available_sizes': [[[150, 111], 'assets/ima...",[{'external_url': 'http://www.brightwire.com/c...,[]
4,52cdef7c4bab8bd67529834c,Los Angeles Times Media Group,los-angeles-times-media-group,http://www.crunchbase.com/company/los-angeles-...,http://www.latimes.com,http://www.latimes.com/blogs,http://feeds.latimes.com/latimes/news,latimes,web,10000,...,"[{'funding_round': {'round_code': 'a', 'source...","{'price_amount': None, 'price_currency_code': ...",[],"[{'description': '', 'address1': '', 'address2...","[{'id': 1530, 'description': 'February 1, 1873...",None,[],[],[],[]


### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [139]:
comps = collection.find({"$and": [{"acquisition.price_amount": {"$gt":10000000}}, {"acquisition.price_currency_code": "EUR"}]})
                             
pd.DataFrame(comps).head()



,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,investments,acquisition,acquisitions,offices,milestones,ipo,video_embeds,screenshots,external_links,partners
0,52cdef7c4bab8bd675297f02,ZYB,zyb,http://www.crunchbase.com/company/zyb,http://zyb.com,http://blog.zyb.com/,http://blog.zyb.com/index.php/feed/,None,mobile,25.0,...,[],"{'price_amount': 31500000, 'price_currency_cod...","[{'price_amount': None, 'price_currency_code':...","[{'description': None, 'address1': 'Esromgade ...",[],None,[],[],[],[]
1,52cdef7d4bab8bd675298bf3,Apertio,apertio,http://www.crunchbase.com/company/apertio,http://www.apertio.com,,,None,mobile,NaN,...,[],"{'price_amount': 140000000, 'price_currency_co...",[],"[{'description': None, 'address1': '', 'addres...",[],None,[],[],[],[]
2,52cdef7d4bab8bd675298f47,Greenfield Online,greenfield-online,http://www.crunchbase.com/company/greenfield-o...,http://www.greenfield.com/content/index.html,,,,public_relations,NaN,...,[],"{'price_amount': 40000000, 'price_currency_cod...","[{'price_amount': 154000000, 'price_currency_c...","[{'description': '', 'address1': '21 River Rd'...",[],None,[],"[{'available_sizes': [[[150, 87], 'assets/imag...",[{'external_url': 'http://www.pr-inside.com/gr...,[]
3,52cdef7e4bab8bd67529a536,Webedia,webedia,http://www.crunchbase.com/company/webedia,http://purepeople.com,,http://www.purepeople.com/rss,purepeople,web,50.0,...,[],"{'price_amount': 70000000, 'price_currency_cod...",[],"[{'description': '', 'address1': '', 'address2...",[],None,[],"[{'available_sizes': [[[150, 93], 'assets/imag...",[],[]
4,52cdef7e4bab8bd67529a729,Wayfinder,wayfinder,http://www.crunchbase.com/company/wayfinder,http://www.wayfinder.com,http://www.wayfinder.com/?id=3848&lang=en-US,http://www.wayfinder.com/?tid=customrssfeeder&...,None,None,NaN,...,[],"{'price_amount': 24000000, 'price_currency_cod...",[],"[{'description': '', 'address1': '', 'address2...",[],None,[],[],[],[]


### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [150]:
trimester = collection.find({"acquisition.acquired_month": 
                             {"$gte":1} and {"$lte": 4}},
                             {"name": 1, "acquisition":1, "_id":0}).limit(10)
                                                                                                             
pd.DataFrame(trimester).head()

,name,acquisition
0,StumbleUpon,"{'price_amount': 29000000, 'price_currency_cod..."
1,Kyte,"{'price_amount': None, 'price_currency_code': ..."
2,Veoh,"{'price_amount': None, 'price_currency_code': ..."
3,Jingle Networks,"{'price_amount': 62500000, 'price_currency_cod..."
4,NetRatings,"{'price_amount': 327000000, 'price_currency_co..."


# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [151]:
foundation = collection.find({"$and": [{"founded_year": 
                             {"$gte":2000} and {"$lte": 2010}}, {"acquisition.acquired_year": {"$gt": 2011}}]},
                             {"name": 1, "acquisition":1, "_id":0})
                                                                                                             
pd.DataFrame(foundation).head()

,name,acquisition
0,Wetpaint,"{'price_amount': 30000000, 'price_currency_cod..."
1,Digg,"{'price_amount': 500000, 'price_currency_code'..."
2,Geni,"{'price_amount': None, 'price_currency_code': ..."
3,blogTV,"{'price_amount': None, 'price_currency_code': ..."
4,Revision3,"{'price_amount': 30000000, 'price_currency_cod..."


### 20. All the companies that have been 'deadpooled' after the third year.

In [153]:
deadpool = collection.find({"deadpooled_year": {"$gt": 3 }},{"name": 1, "deadpooled_year":1, "_id":0})
                                                  
pd.DataFrame(deadpool).head()

,name,deadpooled_year
0,Omnidrive,2008
1,Babelgum,2013
2,Sparter,2008
3,Thoof,2013
4,Mercora,2008
